<h1>Import Packages</h1>

In [14]:
!pip install openai pymupdf

<h1>Upload Resume</h1>

In [15]:
from google.colab import files
import fitz  # pymupdf

print("📄 Upload your resume (PDF format)")
uploaded_resume = files.upload()
resume_path = list(uploaded_resume.keys())[0]

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

resume_text = extract_text_from_pdf(resume_path)
print("Resume text extracted successfully!")

📄 Upload your resume (PDF format)


Saving Moroney updated Resume 202503.pdf to Moroney updated Resume 202503 (1).pdf
Resume text extracted successfully!


In [16]:
from openai import OpenAI
# Replace with own API key
api_key =
client = OpenAI(api_key=api_key)

In [17]:
import requests

def search_jobs(title="software engineer", location="seattle"):
    url = "https://jsearch.p.rapidapi.com/search"

    querystring = {
        "query": f"{title} in {location}",
        "page": "1",           # pagination available
        "num_pages": "1"
    }

    # Replace with own API key
    headers = {
        "x-rapidapi-key": ,
        "x-rapidapi-host": "jsearch.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    if response.status_code == 200:
        return response.json().get("data", [])
    else:
        print("Error:", response.status_code)
        return []

In [18]:
import re

def extract_score(response_text):
    match = re.search(r"(\d{1,3})", response_text)
    return int(match.group(1)) if match else 0

def match_jobs(resume, jobs):
    results = []
    for job in jobs:
        job_desc = job.get("job_description", "")
        prompt = f"""Given the resume:
---
{resume}

Evaluate its fit for the job below and rate from 0–100. Explain briefly:
---
{job_desc}
"""
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        content = response.choices[0].message.content
        score = extract_score(content)
        results.append({**job, "score": score, "reason": content})
    return sorted(results, key=lambda x: x['score'], reverse=True)

def generate_cover_letter(resume, job_description):
    prompt = f"""Write a short, enthusiastic, tailored cover letter using the resume below for the following job.
Resume:
---
{resume}

Job Description:
---
{job_description}
"""
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content


In [19]:
# Customize your job search here
search_title = "software engineer"
search_location = "seattle"

print(f"🔍 Searching for: {search_title} in {search_location}...")
job_listings = search_jobs(search_title, search_location)
print(f"✅ Found {len(job_listings)} jobs")
top_matches = match_jobs(resume_text, job_listings[:5])  # adjust number as needed

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"📍 Location: {job['job_city']}, {job['job_country']}")
    print(f"🔢 Match Score: {job['score']}")
    print(f"🗣 Reason: {job['reason']}")
    print("\n📝 Cover Letter:\n")
    cover_letter = generate_cover_letter(resume_text, job['job_description'])
    print(cover_letter)
    print("\n" + "="*80 + "\n")

🔍 Searching for: software engineer in seattle...
✅ Found 10 jobs

📌 Job: Senior Software Engineer, EX - Seattle at Qualtrics
📍 Location: Seattle, US
🔢 Match Score: 95
🗣 Reason: Rating: 95

Chris Moroney's resume demonstrates a strong fit for the Senior Software Engineer role at Qualtrics. He brings a solid background in full-stack development, cloud infrastructure, and machine learning. The role requires at least 5 years of experience and Chris has around 3 years of experience but has shown advancement in responsibilities and skills in his career progression that could offset the two years gap.

Chris has worked with cross-functional teams as evident from the leadership roles he played at Milliman, aligning technical execution with business needs, and delegating work to interns and lower-level software engineers. He speaks to being curious, a continuous learner, and versatile in his skills - all qualities that the role requires.

Also, he has several certifications in AI and machine le

In [20]:
# 📌 Authenticate with Google & set up APIs
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaInMemoryUpload
import datetime

creds, _ = default()
gc = gspread.authorize(creds)
docs_service = build('docs', 'v1', credentials=creds)
drive_service = build('drive', 'v3', credentials=creds)

In [21]:
# 📄 Save cover letter to Google Doc and return shareable link
def save_cover_letter_to_doc(title, content, folder_name="Generated Cover Letters"):
    # 1. Search for the folder (create it if not found)
    folder_id = None
    response = drive_service.files().list(
        q=f"mimeType='application/vnd.google-apps.folder' and name='{folder_name}' and trashed = false",
        spaces='drive',
        fields='files(id, name)',
    ).execute()
    folders = response.get('files', [])
    if folders:
        folder_id = folders[0]['id']
    else:
        folder_metadata = {
            'name': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive_service.files().create(body=folder_metadata, fields='id').execute()
        folder_id = folder.get('id')

    # 2. Create the doc
    doc = docs_service.documents().create(body={"title": title}).execute()
    doc_id = doc.get("documentId")

    # 3. Move it to the folder
    drive_service.files().update(
        fileId=doc_id,
        addParents=folder_id,
        removeParents='root',
        fields='id, parents'
    ).execute()

    # 4. Write content to doc
    requests = [{"insertText": {"location": {"index": 1}, "text": content}}]
    docs_service.documents().batchUpdate(documentId=doc_id, body={"requests": requests}).execute()

    return f"https://docs.google.com/document/d/{doc_id}/edit"


In [22]:
# 📊 Get or create Google Sheet to track job applications
def get_or_create_sheet(sheet_name="Job Tracker"):
    try:
        sheet = gc.open(sheet_name)
    except:
        sheet = gc.create(sheet_name)
        sheet.share('', perm_type='anyone', role='writer')
        sheet.sheet1.append_row(["Date", "Job Title", "Company", "Location", "Score", "Doc Link"])
    return sheet.sheet1

In [23]:
import datetime
import pytz

# ➕ Add a row to Google Sheet
def log_to_sheet(sheet, job, score, doc_link):
    # Get current time in Pacific Time
    pacific = pytz.timezone('US/Pacific')
    now_pacific = datetime.datetime.now(pacific).strftime("%Y-%m-%d")

    row = [
        now_pacific,
        job['job_title'],
        job['employer_name'],
        f"{job['job_city']}, {job['job_country']}",
        score,
        doc_link
    ]
    sheet.append_row(row)


In [24]:
# ✅ Run this to generate cover letters and log to Google Docs/Sheets
sheet = get_or_create_sheet("Job Tracker")

# Get and sort matched jobs ONCE
sorted_matches = sorted(top_matches, key=lambda x: x['score'], reverse=True)[:5]  # Top 5 consistent picks

for job in top_matches:
    print(f"\n📌 Job: {job['job_title']} at {job['employer_name']}")
    print(f"🔢 Match Score: {job['score']}")

    # Save to Google Doc
    doc_link = save_cover_letter_to_doc(f"{job['job_title']} - {job['employer_name']}", cover_letter)

    # Log everything to Google Sheet
    log_to_sheet(sheet, job, job['score'], doc_link)

    print(f"✅ Saved cover letter: {doc_link}")
    print("=" * 80)



📌 Job: Senior Software Engineer, EX - Seattle at Qualtrics
🔢 Match Score: 95
✅ Saved cover letter: https://docs.google.com/document/d/18kpGvbs34ZeLdsE8bu7HJiZoVYHW9lh5W54uHEx9hZo/edit

📌 Job: Software Engineer Intern at Docusign
🔢 Match Score: 90
✅ Saved cover letter: https://docs.google.com/document/d/1w2mQ9QfGy9Kx4P6djxVku8v1ppkOJxVlZ0fbUVqN6yA/edit

📌 Job: Senior Software Engineer, Backend and Infrastructure (Seattle, WA) at ByteDance
🔢 Match Score: 85
✅ Saved cover letter: https://docs.google.com/document/d/1CNL88Ew_MNAVYPYzwm03l2Gc3pLPyohZtqrxR0FtAus/edit

📌 Job: Software Engineer III, Research Infrastructure at Google
🔢 Match Score: 70
✅ Saved cover letter: https://docs.google.com/document/d/1sZusQjep9Z7r-EkCqJbomik7p9LsgUyNnOQ-a5eEaJw/edit

📌 Job: Software Development Engineer 3 at Adobe
🔢 Match Score: 70
✅ Saved cover letter: https://docs.google.com/document/d/1buReVPbzACDzrCFKVzfcZVErt2z8icqRSVFtI49kRFs/edit
